<a href="https://colab.research.google.com/github/LeherDangal/LeherDangal/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Uploaging data set from kaggle


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:00<00:00, 117MB/s]
100% 80.9M/80.9M [00:00<00:00, 96.5MB/s]


In [ ]:
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset,'r' )as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
  twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Adding coloumn names to the dataset.

In [ ]:
column_names = ['Target', 'ID', 'Date', 'Flag', 'User', 'Text']

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names =column_names, encoding='ISO-8859-1')

In [ ]:
twitter_data.shape


(1600000, 6)

In [ ]:
twitter_data.head()

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


**Data Processing**

Checking for null values

In [ ]:
twitter_data.isnull().sum()

,0
Target,0
ID,0
Date,0
Flag,0
User,0
Text,0


In [ ]:
twitter_data['Target'].value_counts()

,count
Target,
0,800000
4,800000


In [ ]:
twitter_data.replace({'Target':{4:1}}, inplace=True)

In [ ]:
twitter_data['Target'].value_counts()

,count
Target,
0,800000
1,800000


Negative=0

Positive=1

In [ ]:
port_stem=PorterStemmer()

Stemming the data

In [ ]:
def stemming(content):

    stemmed_content = re.sub('{^a-zA-z}',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['Text'].apply(stemming)

In [ ]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          upset can't updat facebook text it... might cr...
2          @kenichan dive mani time ball. manag save 50% ...
3                            whole bodi feel itchi like fire
4          @nationwideclass no, behav all. i'm mad. here?...
                                 ...                        
1599995                       woke up. school best feel ever
1599996    thewdb.com - cool hear old walt interviews! â...
1599997                      readi mojo makeover? ask detail
1599998    happi 38th birthday boo alll time!!! tupac ama...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['Target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: Target, Length: 1600000, dtype: int64


In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['Target'].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' "@hatermagazin i'm in!"
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day! can't wait hear guy store tomorrow!"
 'love wake folgers. bad voic deeper his.']


In [ ]:
print(X_test)

['@mmangen - fine! much time chat twitter. hubbi back summer &amp; tend domin free time.'
 'ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini41 nevertheless hooray! 4700 member wonder safe trip!'
 'feel well' '@supersandro !!!! thank you!']


Converting the text data into numerical form

In [ ]:
vectoriser = TfidfVectorizer()

X_train = vectoriser.fit_transform(X_train)
X_test = vectoriser.transform(X_test)

In [ ]:
print(X_train)

  (0, 548734)	0.272460245511403
  (0, 453116)	0.35795997954657904
  (0, 254267)	0.5248773503338933
  (0, 166830)	0.37842601975344053
  (0, 311959)	0.4213989894780769
  (0, 555967)	0.448720931098355
  (1, 226262)	0.9067376665899747
  (1, 247278)	0.4216951552804088
  (2, 166830)	0.46135196432581543
  (2, 184426)	0.19030000160550456
  (2, 514638)	0.18602125106063966
  (2, 190522)	0.2919883395488319
  (2, 513655)	0.32017162311057684
  (2, 545128)	0.32796751926832923
  (2, 129252)	0.3117944515681845
  (2, 556376)	0.3361458059516176
  (2, 349418)	0.24085594662276874
  (2, 516778)	0.15260594669966598
  (2, 245846)	0.16108271281506445
  (2, 214788)	0.1869628699148727
  (2, 193769)	0.2018446658815102
  (2, 375364)	0.16689112338294135
  (3, 513655)	0.2889371658693274
  (3, 223959)	0.4482843351756085
  (3, 215997)	0.27659430167424054
  :	:
  (1279996, 494789)	0.21762890963383247
  (1279996, 474688)	0.35639996447938693
  (1279996, 315574)	0.5365422013140488
  (1279996, 315576)	0.5521217554769912
 

In [ ]:
print(X_test)

  (0, 55622)	0.17041370659097008
  (0, 74691)	0.1610538837848407
  (0, 117476)	0.26691751430897404
  (0, 163021)	0.37372158998727745
  (0, 193829)	0.2532904390317208
  (0, 200572)	0.23596147172673212
  (0, 238369)	0.2869441563378548
  (0, 354509)	0.449467530296984
  (0, 364065)	0.17667781375470604
  (0, 492691)	0.21841828884993894
  (0, 505258)	0.3459496612358391
  (0, 516778)	0.31514526424889533
  (0, 530649)	0.17887663933344994
  (1, 45607)	0.3039118621705182
  (1, 55622)	0.20816041520057577
  (1, 208893)	0.5712382041014992
  (1, 291622)	0.4070550884371154
  (1, 337825)	0.2840187026690571
  (1, 445343)	0.48174286114843634
  (1, 466635)	0.24404319903374125
  (2, 64354)	0.3536656204135894
  (2, 78469)	0.3786845038674409
  (2, 143688)	0.3624696061947235
  (2, 252255)	0.5874470296410402
  (2, 337915)	0.2587039123196698
  :	:
  (319995, 220218)	0.24721323590915964
  (319995, 286854)	0.23929241469707901
  (319995, 309466)	0.22958307832752411
  (319995, 316463)	0.2041503310854718
  (319995,

Training The Machine Learning Model

Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

accuracy score

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score( Y_train, X_train_prediction)

In [ ]:
print('Accuracy score on training data:', training_data_accuracy)

Accuracy score on training data: 0.81192890625


In [ ]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score( Y_test, X_test_prediction)

In [ ]:
print('Accuracy score on testing data:', testing_data_accuracy)

Accuracy score on testing data: 0.781609375


Model Accuracy = 78%


Saving the trained model

In [ ]:
import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model

In [ ]:
loded_model = pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = loded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

1
[1]
The tweet is Positive
